# Exercice 10

### Q1

Pour une image $u$, son coefficient de Fourier $\hat{u} (k, l)$ correspond au coefficient associé à l'onde pure de vecteur d'onde $\overrightarrow{y} = (k, l)$ dans $u$. Le module de ce coefficient est donc l'amplitude de cette onde dans l'image $u$, et sa phase à la phase de cette onde dans l'image (et est donc reliée à la valeur de l'onde à l'origine, i.e. au "shift" de cette onde le long de son vecteur d'onde)

In [ ]:
import numpy as np
import imtools as im
import matplotlib.pyplot as plt
import cv2

In [ ]:
lena = im.load('lena.pgm').astype('double')
room = im.load('room.pgm').astype('double')
nimes = im.load('nimes.pgm').astype('double')

In [ ]:
def exchange_phase(u: np.ndarray, v: np.ndarray) -> (np.ndarray, np.ndarray):
    """
    Create two new images by exchanging the phases of u and v.
    """
    if not np.array_equal(u.shape, v.shape):
        raise ValueError(f'u and v have different shapes : {u.shape} and {v.shape}')

    u_fft = np.fft.fft2(u)
    v_fft = np.fft.fft2(v)
    u_mod, u_phase = np.abs(u_fft), np.angle(u_fft)
    v_mod, v_phase = np.abs(v_fft), np.angle(v_fft)

    u_swap = np.abs(np.fft.ifft2(v_mod * np.exp(u_phase * 1j)))
    v_swap = np.abs(np.fft.ifft2(u_mod * np.exp(v_phase * 1j)))
    return v_swap, u_swap  # inverted phases

In [ ]:
def phase_swap_plot(u, v):
    u_swap, v_swap = exchange_phase(u, v)
    fig = plt.figure(figsize=(8, 8))

    (subfig1, subfig2) = fig.subfigures(2, 1)
    ax1 = subfig1.subplots(1, 2)
    ax2 = subfig2.subplots(1, 2)

    subfig1.suptitle('Original images')
    ax1[0].imshow(u, cmap='gray')
    ax1[1].imshow(v, cmap='gray')

    subfig2.suptitle('Phase-swapped images')
    ax2[0].imshow(u_swap, cmap='gray')
    ax2[1].imshow(v_swap, cmap='gray')

In [ ]:
phase_swap_plot(lena, room)

Echanger les phases des images revient presque à échanger les images : on voit qu'en échangeant seulement la phase, on retrouve beaucoup de caractéristiques de l'image initiale. La phase semble donc contenir beaucoup d'informations concernant l'apparence de l'image, notamment au niveau des contours et des zones de fort contraste, tandis que le module seul ne contient que peu d'informations, comme on peut le voir ci-dessous.

Ceci semble logique : l'amplitude de l'onde est une information quantitative, tandis que la phase est une information de structure de l'image.

In [ ]:
lena_fft = np.fft.fftshift(np.fft.fft2(lena))
lena_fft_mod = np.abs(lena_fft)
lena_fft_phase = np.exp(np.angle(lena_fft) * 1j)

_, ax = plt.subplots(1, 3, figsize=(18, 6))
ax[0].imshow(lena, cmap='gray')
ax[0].set_title('Original')
ax[1].imshow(np.log(1 + np.abs(np.fft.ifft2(lena_fft_mod))), cmap='gray')  # log to visualize better
ax[1].set_title('IFT of mod(FT)')
ax[2].imshow(np.abs(np.fft.ifft2(lena_fft_phase)), cmap='gray')
ax[2].set_title('IFT of phase(FT)')

Le module seul de la transformée de Fourier de l'image n'est donc absolument pas suffisant pour retrouver l'aspect de l'image initiale, contrairement à la phase.

### Q2

In [ ]:
def randomize_phase(u):
    """
    Return an image with randomized phase (only taking into account the periodic component of the image)
    """

    u_periodic, _ = im.perdecomp(u)
    u_randomized = im.randphase(u_periodic)

    return u_randomized

In [ ]:
arcs = im.load('arcs.png').astype('double')[:, :, 0]
triangles = (im.load('triangles.jpg').astype('double') / 255.0)[:, :, 0]
aperiodic = im.load('aperiodic_12fold.png').astype('double')[:, :, 0]

In [ ]:
_, ax = plt.subplots(1, 3, figsize=(12, 4))
ax[0].imshow(arcs, cmap='gray')
ax[1].imshow(triangles, cmap='gray')
ax[2].imshow(aperiodic, cmap='gray')

In [ ]:
lena_rd = randomize_phase(lena)
arcs_rd = randomize_phase(arcs)
triangles_rd = randomize_phase(triangles)
aperiodic_rd = randomize_phase(aperiodic)

In [ ]:
fig = plt.figure(figsize=(16, 8))

(subfig1, subfig2) = fig.subfigures(2, 1)
ax1 = subfig1.subplots(1, 4)
ax2 = subfig2.subplots(1, 4)

subfig1.suptitle('Original images')
ax1[0].imshow(lena, cmap='gray')
ax1[1].imshow(arcs, cmap='gray')
ax1[2].imshow(triangles, cmap='gray')
ax1[3].imshow(aperiodic, cmap='gray')

subfig2.suptitle('Randomized phase images')
ax2[0].imshow(lena_rd, cmap='gray')
ax2[1].imshow(arcs_rd, cmap='gray')
ax2[2].imshow(triangles_rd, cmap='gray')
ax2[3].imshow(aperiodic_rd, cmap='gray')

In [ ]:
aperiodic_p, aperiodic_s = im.perdecomp(aperiodic)
lena_p, lena_s = im.perdecomp(lena)

In [ ]:
_, ax = plt.subplots(2, 3, figsize=(12, 8))
ax[0][0].imshow(lena, cmap='gray')
ax[0][1].imshow(lena_p, cmap='gray')
ax[0][2].imshow(lena_s, cmap='gray')
ax[1][0].imshow(aperiodic, cmap='gray')
ax[1][1].imshow(aperiodic_p, cmap='gray')
ax[1][2].imshow(aperiodic_s, cmap='gray')

La partie périodique de l'image 'aperiodic' semble égale à un facteur multiplicatif près à l'image originale, tandis que sa partie apériodique est justement presque nulle. Pourtant, cette image est un pavage de Penrose, c'est-à-dire qu'elle est "pseudo-périodique" au sens où des motifs réguliers apparaissent, mais elle n'est pas la répétition d'un motif élémentaire. La partie périodique d'une image est donc difficile à appréhender (d'autant plus que comme on peut le voir pour l'image 'lena', la partie périodique est également quasiment égale à l'image originale)

Comme l'image 'lena', cette image est complètement détruite lorsqu'on rend sa phase aléatoire, ce qui indique qu'elle a bien des propriétés différentes des textures périodiques prises en exemple. Ainsi, les images qui conservent le mieux leur structure originale après que leur phase soit rendue aléatoire sont les textures strictement périodiques.

In [ ]:
rand_smooth = im.randphase(lena_s)
rand_p = im.randphase(lena_p)
rand_full = im.randphase(lena)

_, ax = plt.subplots(1, 3, figsize=(12, 4))
ax[0].imshow(rand_smooth, cmap='gray')
ax[1].imshow(rand_p, cmap='gray')
ax[2].imshow(rand_full, cmap='gray')

In [ ]:
triangles_p, triangles_s = im.perdecomp(triangles)
triangles_rand_p = im.randphase(triangles_p)
triangles_rand_s = im.randphase(triangles_s)
triangles_rand_full = im.randphase(triangles)

_, ax = plt.subplots(1, 3, figsize=(12, 4))
ax[0].imshow(triangles_rand_s, cmap='gray')
ax[1].imshow(triangles_rand_p, cmap='gray')
ax[2].imshow(triangles_rand_full, cmap='gray')

Utiliser la partie non-périodique donne des images différentes de textures à phase aléaoire, on observe en effet une texture très différente en utilisant cette partie. Le module et la phase de la transformée de Fourier de la partie non-périodique sont en effet très structurés. Cette structure doit donc se répercuter dans l'image à phase aléatoire obtenue.

In [ ]:
lena_p_fft = np.fft.fftshift(np.fft.fft2(lena_p))
lena_s_fft = np.fft.fftshift(np.fft.fft2(lena_s))

fig = plt.figure(figsize=(8, 8))
(subfig1, subfig2) = fig.subfigures(2, 1)
ax1 = subfig1.subplots(1, 2)
ax2 = subfig2.subplots(1, 2)

subfig1.suptitle('lena (periodic) FT')
ax1[0].imshow(np.log(1 + np.abs(lena_p_fft)), cmap='gray')
ax1[1].imshow(np.angle(lena_p_fft), cmap='gray')

subfig2.suptitle('lena (smooth) FT')
ax2[0].imshow(np.log(1 + np.abs(lena_s_fft)), cmap='gray')
ax2[1].imshow(np.angle(lena_s_fft), cmap='gray')

### Q3

In [ ]:
n = 512
I = np.arange(-n/2, -n/2 + n - 1)
X, Y = np.meshgrid(I, I)
R = np.hypot(X, Y)
u = 1. * (R < 10)

_, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(u, cmap='gray')
u_rand = im.randphase(u)
ax[1].imshow(u_rand, cmap='gray')

In [ ]:
u_rand_fft = np.fft.fftshift(np.fft.fft2(u_rand))
u_rand_m = np.abs(u_rand_fft)
u_rand_phase = np.angle(u_rand_fft)

_, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(u_rand, cmap='gray')
ax[1].imshow(np.log(1 + u_rand_m), cmap='gray')
ax[2].imshow(u_rand_phase, cmap='gray')

In [ ]:
texture = im.load('texture.pgm').astype('double')

In [ ]:
texture_fft = np.fft.fftshift(np.fft.fft2(texture))
texture_m = np.abs(texture_fft)
texture_phase = np.angle(texture_fft)

_, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(texture, cmap='gray')
ax[1].imshow(np.log(1 + texture_m), cmap='gray')
ax[2].imshow(texture_phase, cmap='gray')

In [ ]:
def draw_ellipse(big_axis, small_axis, theta, center=(0, 0), resolution=1024):
    """
    Draw an ellipse with specified parameters.
    theta is the counter-clockwise angle in degrees.
    """
    image = np.zeros(shape=(resolution, resolution))
    image = cv2.ellipse(image, (image.shape[1]//2 + center[0], image.shape[0]//2 + center[1]),
                        (big_axis, small_axis), -theta, 0, 360, 255, -1)
    return image

In [ ]:
ellipse_1 = draw_ellipse(100, 50, 0)
ellipse_2 = draw_ellipse(100, 50, 15)
ellipse_3 = draw_ellipse(100, 50, 30)
ellipse_4 = draw_ellipse(100, 50, 45)
ellipse_5 = draw_ellipse(100, 50, 60)
ellipse_6 = draw_ellipse(150, 50, 0)
ellipse_7 = draw_ellipse(200, 50, 0)
ellipse_8 = draw_ellipse(200, 100, 0)  # twice as big as ellipse_1

In [ ]:
ellipses = [
    ellipse_1,
    ellipse_2,
    ellipse_3,
    ellipse_4,
    ellipse_5,
    ellipse_6,
    ellipse_7,
    ellipse_8,
]

n_el = len(ellipses)
_, ax = plt.subplots(n_el, 2, figsize=(2 * 4, 3 * n_el))

for i, el in enumerate(ellipses):
    ax[i][0].imshow(el, cmap='gray')
    el_fft = np.fft.fftshift(np.fft.fft2(el))
    ax[i][1].imshow(np.log(1 + np.abs(el_fft)[482:542, 482:542]), cmap='gray')

In [ ]:
im.View(np.log(1 + np.abs(np.fft.fftshift(np.fft.fft2(ellipse_2)))))

Les longueurs des axes de l'ellipse initiale et de celles visualisées pour le module de la transformée de Fourier semblent suivrent une dépendence en inverse : la longueur du grand axe de l'ellipse initiale est inversement proportionnelle à celle du *petit* axe de l'ellipse dans le domaine de Fourier, et de même pour le petit axe. Ceci est plus clair lorsqu'on visualise les ellipses 1, 7 et 8 :

In [ ]:
el_178 = [
    ellipse_1,
    ellipse_7,
    ellipse_8,
]

n_el = len(el_178)
_, ax = plt.subplots(n_el, 2, figsize=(2 * 4, 3 * n_el))

for i, el in enumerate(el_178):
    ax[i][0].imshow(el, cmap='gray')
    el_fft = np.fft.fftshift(np.fft.fft2(el))
    ax[i][1].imshow(np.log(1 + np.abs(el_fft)[482:542, 482:542]), cmap='gray')

In [ ]:
im.View(np.log(1 + np.abs(np.fft.fftshift(np.fft.fft2(ellipse_3)))))

On voit bien que l'ellipse obtenue dans le domaine du Fourier est deux fois plus petite pour l'ellipse 8 que l'ellipse 1, et l'ellipse 8 est deux fois plus grande. Les tailles exactes des axes sont :

* Ellipse 1 :
* * Image : grand-axe = 100px, petit-axe = 50px
* * Fourier : grand-axe = 24px, petit-axe = 12px (N.B. : valeurs pour la plus petite ellipse, au centre du domaine)
* Ellipse 7 :
* * Image : grand-axe = 200px, petit-axe = 50px
* * Fourier : grand-axe = 24px, petit-axe = 6px
* Ellipse 8 :
* * Image : grand-axe = 200px, petit-axe = 100px
* * Fourier : grand-axe = 12px, petit-axe = 6px

Le ratio $\dfrac{\text{grand-axe}}{\text{petit-axe}}$ est donc préservé entre l'image réelle et le domaine de Fourier (on peut s'en assurer en prenant des mesures sur de multiples ellipses dans le domaine de Fourier), de plus la dépendence est bien inversement linéaire entre les deux domaines.
On a donc, en notant $a$ et $b$ les grand et petit-axes de l'ellipse initiale, et $\hat{a}$, $\hat{b}$ les grand et petit-axes de l'ellipse dans le domaine de Fourier, pour une résolution de $1024\times 1024$ :
\begin{equation*}
\begin{split}
\frac{a}{1024}\cdot \frac{\hat{b}}{1024} & = 1.144 \, 10^{-3} = C\\
\frac{b}{1024\cdot \frac{\hat{a}}{1024} & = C
\end{split}
\end{equation*}

(On divise par la résolution pour exprimer les longueurs en proportion de la taille d'un pixel)

Pour ce qui est de l'orientation de l'ellipse dans le domaine de Fourier, elle semble simplement être tournée de 90° dans le sens trigonométrique. On peut vérifier ceci à l'aide des ellipses 1, 2 et 3 :

* Ellipse 1 :
* * Image : $\theta_I = 0 \text{ rad}$  (N.B. angle entre le demi-grand axe et l'axe $x$ (sens positif), sens trigonométrique)
* * Fourier : $\theta_F = \frac{\pi}{2} \text{ rad} = \theta_I + \frac{\pi}{2} \text{ rad}$
* Ellipse 2 :
* * Image : $\theta_I = 15° = \frac{\pi}{12} \text{ rad}$
* * Fourier : $\theta_F = 108° \approx \theta_I + \frac{\pi}{2} \text{ rad}$
* Ellipse 3 :
* * Image : $\theta_I = 30° = \frac{\pi}{6} \text{ rad}$
* * Fourier : $\theta_F = 122° \approx \theta_I + \frac{\pi}{2} \text{ rad}$

Cette règle semble donc vérifiée à 2 ou 3 degrés près. En appliquant ceci à l'image obtenue en visualisant le module de la transformée de Fourier de la texture, on obtient :
* Grand axe : 22.02px
* Petit axe : 15.62px
* $\theta_F = 125.34°$

On estime donc que l'ellipse initiale a les caractéristiques suivantes :
* Petit axe : $\frac{500^2C}{22.02} = 13\text{px}$
* Grand axe : $\frac{500^2C}{15.62} = 18\text{px}$
* $\theta_I = 35.34°$

In [ ]:
unknown_ellipse = draw_ellipse(18, 13, 35.34, resolution=500)

In [ ]:
_, ax = plt.subplots(1, 3, figsize=(12, 4))
ax[0].imshow(unknown_ellipse, cmap='gray')
ax[0].set_title('Ellipse originale estimée')
ax[1].imshow(np.log(1 + np.abs(np.fft.fftshift(np.fft.fft2(unknown_ellipse)))), cmap='gray')
ax[1].set_title('FT de l\'ellipse estimée')
ax[2].imshow(np.log(1 + texture_m), cmap='gray')
ax[2].set_title('FT de l\'ellipse originale')

In [ ]:
im.View(np.log(1 + np.abs(np.fft.fftshift(np.fft.fft2(unknown_ellipse)))))

L'orientation semble correcte à quelques degrés près, ainsi que le rapport des longueurs des axes, mais pas les longueurs des axes elles-mêmes : la longueur du grand axe de l'ellipse formée sur la transformée de Fourier de l'ellipse estimée est trop grande, d'un facteur 1.45. Appliquons ce facteur à l'ellipse originale estimée

In [ ]:
unknown_ellipse_corr = draw_ellipse(round(1.45 * 18, ndigits=None), round(13 * 1.45, ndigits=None), 35.34, resolution=500)
_, ax = plt.subplots(1, 3, figsize=(12, 4))
ax[0].imshow(unknown_ellipse_corr, cmap='gray')
ax[0].set_title('Ellipse originale estimée')
ax[1].imshow(np.log(1 + np.abs(np.fft.fftshift(np.fft.fft2(unknown_ellipse_corr)))), cmap='gray')
ax[1].set_title('FT de l\'ellipse estimée')
ax[2].imshow(np.log(1 + texture_m), cmap='gray')
ax[2].set_title('FT de l\'ellipse originale')

On se rapporche de l'ellipse originale, mais on a encore une erreur visible d'orientation et de longueurs d'axes. Le modèle appliqué ici n'est donc pas complet, ou il y a peut-être un rôle important de la résolution non pris en compte dans les estimations faites ici.